# 「Python による経済・経営分析のための実践的データサイエンス」第3章(pp.61-71)

必要なパッケージをインポートする

In [1]:
#追加部分ここから
# FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
# の出力を抑止する。
import warnings
warnings.simplefilter('ignore', FutureWarning)
#追加部分ここまで

In [2]:
#RESAS を API で利用する
#利用するパッケージをインポートする
#参考文献; https://note.nkmk.me/python-resas-api-download/

import json
import urllib.request
import pprint
import pandas as pd

API キーをインポートする

In [3]:
#API キーを取得し, 読み込めるか確認する
#API キーは、https://opendata.resas-portal.go.jp/　にアクセスして、利用登録することで発行されます。
#ファイルはutf-8 でエンコーディングすること。
#追加部分：もしBOM付きのutf-8なら、encodeing = 'utf_8_sig'とします。
with open('api_key.json', mode = 'r', encoding = 'utf-8') as f:
    api_key = json.load(f)
    
#中身が取り出せることが確認できる
# {"X-API-KEY":"xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

#放り込んだデータの中身f を確認する   
print(f)

#json.load で取り出した api_key の中身を確認する
print(api_key)
print(type(api_key))

<_io.TextIOWrapper name='api_key.json' mode='r' encoding='utf-8'>
{'X-API-KEY': 'x9StWoRYaN38DXYpKHzqMVcuZBDfesU1yLrLS2hN'}
<class 'dict'>


東京23区+市区町村の年単位の創業比率をもとめる

In [4]:
# api_key.json に指定したAPI キーを読み込む
#ファイルはutf-8 でエンコーディングすること
#追加部分：もしBOM付きのutf-8なら、encodeing = 'utf_8_sig'とします。
with open('api_key.json', mode = 'r', encoding = 'utf-8') as f:
    api_key = json.load(f)

#東京都の市町村コードを取得する

url_base = 'https://opendata.resas-portal.go.jp/api/v1/cities'

p = {'prefCode': 13}
url = url_base + '?' + urllib.parse.urlencode(p)
req = urllib.request.Request(url, headers=api_key)

with urllib.request.urlopen(req) as response:
    data = response.read()
    
d = json.loads(data.decode())
#print(d)
tokyo_list=pd.json_normalize(d['result'],  meta=['bigCityFlag', 'cityCode', 'cityName', 'prefCode'])

#東京23区+市区町村の年単位の創業比率をRESAS API から取得する

url_base = 'https://opendata.resas-portal.go.jp/api/v1/municipality/foundation/perYear'

#空のDataFrame を用意する
s=pd.DataFrame()

#東京都のデータを取得する
for j in tokyo_list.cityCode:
    p = {'prefCode': 13, 'cityCode': j}
    url = url_base + '?' + urllib.parse.urlencode(p)
    #print(url)
# https://opendata.resas-portal.go.jp/api/v1/municipality/taxes/perYear?prefCode=13&cityCode=13103

    req = urllib.request.Request(url, headers=api_key)

    with urllib.request.urlopen(req) as response:
        data = response.read()

    d = json.loads(data.decode())
    #pprint.pprint(d)
    
    #各市区町村ごとのデータをJSON からDataframe 形式に変換し, k に放り込む
    k=pd.json_normalize(d['result'], record_path='data', meta=['cityCode', 'cityName', 'prefCode', 'prefName'])
    #print(k)
    
    #k の情報をs に放り込む
    s=s.append(k)
    #これを市区町村分繰り返す

#東京都のデータをまとめて出力する
print(s)

          year  value cityCode cityName prefCode prefName
0   2001～2004年   8.58    13101     千代田区       13      東京都
1   2004～2006年   9.59    13101     千代田区       13      東京都
2   2006～2009年   3.69    13101     千代田区       13      東京都
3   2009～2012年   2.70    13101     千代田区       13      東京都
4   2012～2014年  11.18    13101     千代田区       13      東京都
..         ...    ...      ...      ...      ...      ...
1   2012～2014年   0.00    13402     青ヶ島村       13      東京都
2   2014～2016年   0.00    13402     青ヶ島村       13      東京都
0   2009～2012年   3.84    13421     小笠原村       13      東京都
1   2012～2014年   5.86    13421     小笠原村       13      東京都
2   2014～2016年   5.96    13421     小笠原村       13      東京都

[333 rows x 6 columns]


全国の市町村コードを取得する

In [5]:
#全国の市町村コードを取得する

url_base = 'https://opendata.resas-portal.go.jp/api/v1/cities'
city_list=pd.DataFrame()

for i in range(1,48):
    p = {'prefCode': i}
    url = url_base + '?' + urllib.parse.urlencode(p)
    #print(url)
    req = urllib.request.Request(url, headers=api_key)

    with urllib.request.urlopen(req) as response:
        data = response.read()

    d = json.loads(data.decode())
    #pprint.pprint(d, width=100)
    city_list2=pd.json_normalize(d['result'],  meta=['bigCityFlag', 'cityCode', 'cityName', 'prefCode'])
    city_list=city_list.append(city_list2)

print(city_list.cityCode)

0     01100
1     01101
2     01102
3     01103
4     01104
      ...  
36    47361
37    47362
38    47375
39    47381
40    47382
Name: cityCode, Length: 1922, dtype: object


県コードと市区町村コードをプロットする

In [6]:
for a,b in zip(city_list.prefCode, city_list.cityCode):
    print(a,b)

1 01100
1 01101
1 01102
1 01103
1 01104
1 01105
1 01106
1 01107
1 01108
1 01109
1 01110
1 01202
1 01203
1 01204
1 01205
1 01206
1 01207
1 01208
1 01209
1 01210
1 01211
1 01212
1 01213
1 01214
1 01215
1 01216
1 01217
1 01218
1 01219
1 01220
1 01221
1 01222
1 01223
1 01224
1 01225
1 01226
1 01227
1 01228
1 01229
1 01230
1 01231
1 01233
1 01234
1 01235
1 01236
1 01303
1 01304
1 01331
1 01332
1 01333
1 01334
1 01337
1 01343
1 01345
1 01346
1 01347
1 01361
1 01362
1 01363
1 01364
1 01367
1 01370
1 01371
1 01391
1 01392
1 01393
1 01394
1 01395
1 01396
1 01397
1 01398
1 01399
1 01400
1 01401
1 01402
1 01403
1 01404
1 01405
1 01406
1 01407
1 01408
1 01409
1 01423
1 01424
1 01425
1 01427
1 01428
1 01429
1 01430
1 01431
1 01432
1 01433
1 01434
1 01436
1 01437
1 01438
1 01452
1 01453
1 01454
1 01455
1 01456
1 01457
1 01458
1 01459
1 01460
1 01461
1 01462
1 01463
1 01464
1 01465
1 01468
1 01469
1 01470
1 01471
1 01472
1 01481
1 01482
1 01483
1 01484
1 01485
1 01486
1 01487
1 01511
1 01512
1 01513


全国市区町村+東京23区の年単位の企業数

In [7]:
#全国市区町村+東京23区の年単位の企業数をRESAS API から取得する
#マニュアルは https://opendata.resas-portal.go.jp/docs/api/v1/municipality/company/perYear.html を参照のこと

url_base = 'https://opendata.resas-portal.go.jp/api/v1/municipality/company/perYear'

#空のDataFrame を用意する
s=pd.DataFrame()
k=pd.DataFrame()

#全都道府県のデータを取得する
#zip 関数を使って, prefCode と cityCode ごとに情報を読み込む
for a,b in zip(city_list.prefCode, city_list.cityCode):
    
    #各市区町村ごとのデータをJSON からDataframe 形式に変換し, k に放り込む
    #データが取得できない市町村はスキップするために, try-except-else 構文を使う
    #追加部分：TypeErrorだけでなく、NotImplementedErrorになる場合があるので、すべてのerrorを対象とする
    try:
        #都道府県コードと市町村コードを変数として放り込む
        p = {'prefCode': a, 'cityCode': b}
        url = url_base + '?' + urllib.parse.urlencode(p)
        #print(url)
        # https://opendata.resas-portal.go.jp/api/v1/municipality/taxes/perYear?prefCode=13&cityCode=13103

        req = urllib.request.Request(url, headers=api_key)

        with urllib.request.urlopen(req) as response:
            data = response.read()

        #JSON 形式でデータを取得する
        d = json.loads(data.decode())
        #pprint.pprint(d)
        #取得したデータを panda 形式にする
        k=pd.json_normalize(d['result'], record_path='data', meta=['cityCode', 'cityName', 'prefCode', 'prefName'])
#    except TypeError:
# 修正部分 TypeErrorだけでなく、すべてのエラーを対象とする
    except:
        print("not available")
    else:
        pass
    
    #print(a,b,k)
    
    #k の情報をs に放り込む
    s=s.append(k)
    #これを市区町村分繰り返す

#全国の市区町村レベルのデータをまとめて出力する
print(s)

not available
not available
not available
not available
not available
not available
    year  value cityCode cityName prefCode prefName
0   2009  55436    01100      札幌市        1      北海道
1   2012  50924    01100      札幌市        1      北海道
2   2014  50755    01100      札幌市        1      北海道
3   2016  47112    01100      札幌市        1      北海道
0   2009  55436    01101      札幌市        1      北海道
..   ...    ...      ...      ...      ...      ...
3   2016    345    47381      竹富町       47      沖縄県
0   2009    215    47382     与那国町       47      沖縄県
1   2012    184    47382     与那国町       47      沖縄県
2   2014    186    47382     与那国町       47      沖縄県
3   2016    159    47382     与那国町       47      沖縄県

[7681 rows x 6 columns]


In [8]:
#最後にデータをcsvとして出力しましょう
s.to_csv("firm_num.csv")